In [ ]:
# Do our imports for the code
from google.cloud import bigquery
from google.oauth2 import service_account
import random

In [ ]:
#connect to UMT GBQ instance. 
service_path = 'C:\\Users\\markmcdiffett\\Desktop\\2021fallsemester\\Applieddataanalytics\\'
service_file = 'umt-msba-037daf11ee16.json'   
gbq_proj_id = 'umt-msba'
gbq_dataset_id = 'transactions'


private_key =service_path + service_file



In [ ]:
# Get your credentials
credentials = service_account.Credentials.from_service_account_file(service_path + service_file)

# And create a client to talk to GBQ
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)

In [ ]:
#create sample of owners
#query finding all card numbers

qry = """
SELECT
  card_no
FROM
  `umt-msba.transactions.transArchive*`
WHERE CARD_NO != 3
group by card_no
order by card_no
"""
# perform query
query_job = client.query(
    qry,
    location="US",
)

#create full list of owners
owners_list = []
for row in query_job:
    owners_list.append(int(row[0]))

#create sampling of 400 owners and format for query
owners_list = random.choices(owners_list, k = 400)
",".join([str(item) for item in owners_list])
  

In [ ]:
# query the sample of owners 

qry2_start = """
SELECT
  *
FROM
  `umt-msba.transactions.transArchive*`
WHERE CARD_NO in ("""

qry2_end = ") ORDER BY card_no"

#insert list of owner sample into query
qry2 = qry2_start + ",".join([str(item) for item in owners_list]) + qry2_end

# And we execute queries with `client.query`
query_job2 = client.query(
    qry2,
    location="US",
)



In [ ]:
#create text file of sample records
with open("sample_owner_records.txt","w") as outfile :
    for row in query_job2 : 
        outfile.write("\t".join([str(item) for item in row]) + "\n")